# Deploy the ML model which was already trained

### Install necessary packages

In [1]:
!pip install gradio ultralytics opencv-python > /dev/null
!pip install ultralytics > /dev/null
!pip install opencv-python torch deep_sort_realtime
!pip install git+https://github.com/KaiyangZhou/deep-person-reid.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.8 MB/s eta 0:00:00
  Cloning https://github.com/KaiyangZhou/deep-person-reid.git to /tmp/pip-req-build-u43m739n
  Running command git clone --filter=blob:none --quiet https://github.com/KaiyangZhou/deep-person-reid.git /tmp/pip-req-build-u43m739n
  Resolved https://github.com/KaiyangZhou/deep-person-reid.git to commit 566a56a2cb255f59ba75aa817032621784df546a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
  Created wheel for torchreid: filename=torchreid-1.4.0-cp310-cp310

### Import necessary libraries

In [2]:
import os
import cv2
import random
import tempfile
import gradio as gr
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Function to load model

In [4]:
def load_model(train_folder, pre_trained_model, pre_trained=False):
    ROOT_DIR = "/content/gdrive/MyDrive/ie643-1/data"
    if not pre_trained:
        model = YOLO(os.path.join(ROOT_DIR, 'runs', 'detect', f'{train_folder}', 'weights', 'best.pt'))
    else:
        model = YOLO(pre_trained_model)
    return model

### Helper function to draw poses

In [5]:
POSE_PAIRS = [
    (0, 1), (0, 2), (1, 2), (1, 3), (2, 4),
    (3, 5), (4, 6), (5, 7), (7, 9), (5, 6), (6, 8), (8, 10),
    (5, 11), (6, 12), (11, 13), (13, 15), (11, 12), (12, 14), (14, 16)
]

def get_color_for_pair(pair):
    COLOR_GROUPS = {
        "group1": (0, 0, 255),
        "group2": (0, 255, 0),
        "group3": (255, 0, 0),
        "group4": (255, 255, 0),
        "group5": (255, 255, 255)
    }

    # Points in each group
    group1_points = {0, 1, 2, 3, 4}
    group2_points = {5, 6, 7, 8, 9, 10}
    group3_points = {11, 12, 13, 14, 15, 16}
    group4_points = {5, 6, 11, 12}
    group5_points = {3, 4, 5, 6}

    # Check pair group
    if pair[0] in group1_points and pair[1] in group1_points:
        return COLOR_GROUPS["group1"]
    elif pair[0] in group2_points and pair[1] in group2_points:
        return COLOR_GROUPS["group2"]
    elif pair[0] in group3_points and pair[1] in group3_points:
        return COLOR_GROUPS["group3"]
    elif pair[0] in group4_points and pair[1] in group4_points:
        return COLOR_GROUPS["group4"]
    elif pair[0] in group5_points and pair[1] in group5_points:
        return COLOR_GROUPS["group5"]
    else:
        print("NO")
        return (255, 255, 255)


### Function to process the video

In [12]:
def process_video(video_path, model):
    tracker = DeepSort(max_age=30,n_init=10,nn_budget=200,nms_max_overlap=0.3,max_iou_distance=0.6,max_cosine_distance=0.3,embedder='torchreid',embedder_model_name='osnet_x1_0')
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    temp_video_output = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
    model1 = YOLO("yolo11n-pose.pt")
    out = cv2.VideoWriter(temp_video_output.name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))
    unique_ids = set()
    while cap.isOpened():
        detected_bodies = []
        success, frame = cap.read()
        if success:
            # Run YOLO tracking on the frame
            results = model(frame)
            annotated_frame = frame.copy()  # Start with the original frame
            results1 = model1(frame)
            detections1 = results1[0]
            if hasattr(detections1, 'keypoints') and detections1.keypoints is not None:
                # keypoint_connections = detections1.keypoints.connections
                for person_keypoints in detections1.keypoints.data:
                    # Draw keypoints as circles
                    points = []
                    num = -1
                    for point in person_keypoints:
                        num = num + 1
                        if len(point) == 3:  # Ensure the format is x, y, confidence
                            x, y, confidence = point
                            if confidence > 0.5:  # Confidence threshold
                                cv2.circle(annotated_frame, (int(x), int(y)), 5, (0, 0, 0), -1)  # Default green
                                points.append((int(x), int(y)))
                            else:
                                points.append(None)
                        else:
                            points.append(None)

                    # # Draw lines connecting keypoints
                    for start_idx, end_idx in POSE_PAIRS:
                        if start_idx < len(points) and end_idx < len(points) and points[start_idx] and points[end_idx]:
                            color = get_color_for_pair((start_idx, end_idx))
                            cv2.line(annotated_frame, points[start_idx], points[end_idx], color, 2)

            for result in results[0].boxes:
                # print(result.xyxy[0])
                x1, y1, x2, y2 = map(int, result.xyxy[0])  # Get bounding box coordinates
                conf = result.conf.item()  # Confidence score
                class_id = int(result.cls.item())  # Class ID

                if class_id == 1 and conf>=0.6:  # Class ID for body (assuming 1 is for 'person')
                    detected_bodies.append([(x1, y1, x2-x1, y2-y1), conf,class_id])  # Append body detection

            # Pass detections to Deep SORT for tracking
            tracks = tracker.update_tracks(detected_bodies, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue
                #print(track)
                bbox = track.to_tlbr()  # Bounding box
                track_id = track.track_id
                unique_ids.add(track_id)
                # Draw bounding box and track ID
                cv2.rectangle(annotated_frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
                cv2.putText(annotated_frame, f'ID: {track_id}', (int(bbox[0]), int(bbox[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            out.write(annotated_frame)
        else:
            break
    cap.release()
    out.release()
    return temp_video_output.name, len(unique_ids)

### Helper function to detect people

In [13]:
def detect_people_in_video(video):
    pre_trained = False
    train_folder = 'train5'
    pre_trained_model = "yolov11n.pt"
    model = load_model(train_folder, pre_trained_model, pre_trained)
    output_video_path, unique_persons = process_video(video, model)
    return output_video_path, f"{unique_persons}"

## Run to deploy the model

In [14]:
gr_interface = gr.Interface(
    fn=detect_people_in_video,
    inputs=gr.Video(label="Upload a video"),
    outputs=[gr.Video(label="Processed Video"), gr.Textbox(label="Unique Person Count")],
    title="Unique Person Detection",
    description="Upload a video and the our model will detect unique persons and pose."
)
gr_interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93f66c65a56cbab1d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352

0: 384x640 25 Heads, 32 Persons, 65.0ms
Speed: 1.8ms preprocess, 65.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 44.8ms
Speed: 1.2ms preprocess, 44.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 Heads, 33 Persons, 10.9ms
Speed: 2.2ms preprocess, 10.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 11.0ms
Speed: 1.5ms preprocess, 11.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 Heads, 32 Persons, 10.4ms
Speed: 1.7ms preprocess, 10.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 11.7ms
Speed: 1.5ms prep

/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:337: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4.
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://93f66c65a56cbab1d5.gradio.live
